In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
data = pd.read_csv('test_new.csv')

In [3]:
data.head()

,schol_id,University,Major,Level,Language,Start Date,Tuition Covered,Accomodation covered?,Living Expense Covered?,Tuition fees to pay,Original Tuition fee,Accomodation to pay,Living expense to pay
0,1,Shanghai University of Traditional Chinese Med...,Summer School (TCM),Non-Degree,English,"Jun,2020",free,No,No,0,7500,6500-13000/ four weeks ( hotel fee around the ...,About 1000/4 weeks
1,2,Changchun University of Science and Technology,Preparatory Program of Humanities Direction(Pa...,Non-Degree,Chinese,"Sep,2019","10,000",No,No,"10,000","20,000",About 1500/month,About 1500/month
2,3,Changchun University of Science and Technology,Preparatory Program of Business Direction(Path A),Non-Degree,Chinese,"Sep,2019","10,000",No,No,"10,000","20,000",About 1500/month,About 1500/month
3,4,Changchun University of Science and Technology,Preparatory Program of Science and Engineering...,Non-Degree,Chinese,"Sep,2019","10,000",No,No,"10,000","20,000",About 1500/month,About 1500/month
4,5,Qingdao University,Long-term Chinese Language Program,Non-Degree,Chinese,"Sep,2019",7000,yes,yes,0,7000,0,0/month


# Cleaning Requirements

- PART 1 - Convert start date to two columns (Month, Year)
- PART 2 - Convert the contents of the field Tuition Covered
- PART 3 - Convert the contents of the field Accomodation Covered
- PART 4 - Convert the contents of the field Living expense Covered
- PART 5 - Convert tuition fees, original tuition fees to pay to currency (RMB)
- PART 6 - Convert Accomodation to pay to two fields (Accomodation price, Accomodation duration)
- PART 7 - Convert Living expense to pay to two fields (living expense , living duration)

### PART 1 - Convert start date to two columns

In [4]:
i=0
start_month = []
start_year = []
data['Start_Month'] = ""
data['Start_Year'] = ""
for i in range(0,len(data)):
    start_month.append(data['Start Date'][i].split(',')[0])
    start_year.append(data['Start Date'][i].split(',')[1])

In [5]:
data['Start_Month'] = start_month
data['Start_Year'] = start_year

In [6]:
data = data.drop('Start Date',axis=1)

### PART 2 - Convert the contents of the field tuition covered

In [7]:
data['Tuition Covered'].value_counts()

30,000                               210
No                                   178
18,800                               162
20,000                               159
15000-30000                          135
Not more than RMB 25,000 per year    120
7500                                 109
17,500-35,000                        103
free                                  93
15,000-30,000                         85
9,000                                 73
15,000                                69
35,000-42,900                         67
8,000                                 64
Free                                  64
16,800                                64
20000                                 62
10,000                                58
9100-26000                            58
25,000.00-34,000.00                   57
11500-44500                           57
34,000.00-51,000.00                   57
7,800-8,800                           56
15,000-26,000                         55
20000-35000     

In [8]:
tuition_covered_unique = data['Tuition Covered'].value_counts().keys()
count=0
for count in range(2): #Since we'LL be getting some data from tuition_orig, we'll have to repeat the loop
    tuition_covered_unique = data['Tuition Covered'].value_counts().keys()
    for i in tuition_covered_unique:
        try:
            i_copy = i
            i = i.strip()
            #Convert free,Free,Yes to data from original tuition fee
            temp = data['Tuition Covered']
            data.loc[temp=='free','Tuition Covered'] = data['Original Tuition fee']
            data.loc[temp=='Free','Tuition Covered'] = data['Original Tuition fee']
            data.loc[temp=='Yes','Tuition Covered'] = data['Original Tuition fee']

            #Replace No to 0
            data.loc[temp=='No','Tuition Covered'] = 0

            #Replace actual recognized numbers to int
            try:
                data['Tuition Covered'] = data['Tuition Covered'].replace(i, int(i))
            except:
                pass

            #Find numbers with only ',' in it and convert it to int
            try:
                if str(i).find(',')!=-1:
                    if str(i).find('-')==-1:
                        new_i=int(i.replace(',',''))
                        data['Tuition Covered'] = data['Tuition Covered'].replace(i,new_i)
            except:
                temp = re.findall(r'\d+', i) 
                res = list(map(int, temp))
                data['Tuition Covered'] = data['Tuition Covered'].replace(i,int(res[0]))

            #Not more than RMB 25,000 per year to 25000
            data['Tuition Covered'] = data['Tuition Covered'].replace('Not more than RMB 25,000 per year',25000)

            #Replace **-** to the average of both the numbers(No commas included)
            try:
                if str(i).find('-')!=-1:
                    if str(i).find(',')==-1:
                        left_num = i[:str(i).find('-')]
                        right_num = i[str(i).find('-')+1:]
                        average = (int(left_num) + int(right_num))/2
                        data['Tuition Covered'] = data['Tuition Covered'].replace(i,average)
            except:
                pass

            #Replace **-** to the average of both the numbers(commas included)
            try:
                if str(i).find('-')!=-1:
                    if str(i).find(',')!=-1:
                        left_num = i[:str(i).find('-')]
                        left_num = left_num.replace(',','')
                        right_num = i[str(i).find('-')+1:]
                        right_num = right_num.replace(',','')
                        average = (float(left_num) + float(right_num)) /2 
                        data['Tuition Covered'] = data['Tuition Covered'].replace(i_copy,average)
            except:
                pass
            
            #Convert numbers with rmb in the end to int
            try:
                if str(i).find('rmb')!=-1:
                    new_i = int(str(i).replace('rmb',''))
                    data['Tuition Covered'] = data['Tuition Covered'].replace(i,new_i)
            except:
                pass

            #Convert data with 'to' in it to int
            try:
                if str(i).find('to')!=-1:
                    left_num = int(str(i)[:str(i).find('to')])
                    right_num = int(str(i)[str(i).find('to')+2:])
                    average = (left_num + right_num)/2
                    data['Tuition Covered'] = data['Tuition Covered'].replace(i,average)
                    
            except:
                pass
            
        except:
            pass

In [9]:
schol_ids = list(data[data['Tuition Covered']=='24000 8000'].index.values)
data = data.drop(data.index[schol_ids])

In [10]:
data['Tuition Covered'] = pd.to_numeric(data['Tuition Covered'])

### PART 3 - Accomodation Covered

In [11]:
data['Accomodation covered?'].value_counts()

No           2503
Yes           395
yes           211
0             129
no            111
4,000          60
8400.00        57
10000          48
Free           33
12,000.00      29
Name: Accomodation covered?, dtype: int64

In [12]:
accomodation_covered_unique = data['Accomodation covered?'].value_counts().keys()

for i in accomodation_covered_unique:
    #Combine No, no, 0
    if (i=='No'):
        data['Accomodation covered?'] = data['Accomodation covered?'].replace(i,0)
    elif (i=='no'):
        data['Accomodation covered?'] = data['Accomodation covered?'].replace(i,0)
    elif (i=='0'):
        data['Accomodation covered?'] = data['Accomodation covered?'].replace(i,0)
    #Combine everything else to 1
    else:
        data['Accomodation covered?'] = data['Accomodation covered?'].replace(i,1)

In [13]:
data['Accomodation covered?'].value_counts()

0    2743
1     833
Name: Accomodation covered?, dtype: int64

### PART 4 - Convert the contents of the field Living expense Covered

In [14]:
data['Living Expense Covered?'].value_counts()

No                                     2404
Yes                                     382
no                                      156
3000 RMB / month                        121
2000/month                               86
yes                                      85
44,000.00                                57
No\nOther fee about 200                  48
No \nOther fee about 200                 48
0                                        37
1400/month                               30
42,800.00                                29
2500/month                               24
No\nOther fee about 7200                 22
1,000 RMB/month                          21
1800/month                               13
1200 RMB/month                            8
800 RMB/month                             3
2500/month\nMedical Insurance: Free       2
Name: Living Expense Covered?, dtype: int64

In [15]:
living_covered_unique = data['Living Expense Covered?'].value_counts().keys()

In [16]:
for i in living_covered_unique:
    i_upper = str(i).upper()
    if i_upper.find('NO')!=-1:
        data['Living Expense Covered?'] = data['Living Expense Covered?'].replace(i,0)
    else:
        data['Living Expense Covered?'] = data['Living Expense Covered?'].replace(i,1)

In [17]:
data['Living Expense Covered?'].value_counts()

0    2678
1     898
Name: Living Expense Covered?, dtype: int64

### PART 5 - Convert tuition fees, original tuition fees to pay to currency (RMB)

In [18]:
data['Tuition fees to pay'].value_counts()

0                                           895
11000-26000                                 135
0- 32400                                    120
10,000                                      117
0 RMB                                       101
5,000 - 15,000 RMB                           77
0-17,500                                     76
35000                                        76
8,500-12,500                                 69
0-3,000                                      69
0- 11000                                     65
30000                                        64
0-15,000                                     64
0 - 10,400 RMB                               64
0 - 10,400                                   64
0-18000                                      62
0-13000                                      58
0-11500                                      57
7,800-8,800                                  56
1000- 11000                                  55
15000-30000                             

#### A - Tuition Fees

Types of numbers listed
1. integer
2. 50-100
3. 50,000
4. 0-20,000
5. 10,000-20,000
6. 0 RMB
7. 5,000 - 15,000 RMB
8. 0Acco...
9. 100 to 200

In [19]:
tuition_topay_unique = data['Tuition fees to pay'].value_counts().index
for i in tuition_topay_unique:
    data['Tuition fees to pay'] = data['Tuition fees to pay'].replace(str(i),str(i).strip()) #To make sure that the data has no additional white spaces

In [20]:
def remove_commas(i):
    return str(i).replace(',','')
def remove_rmb(i):
    return str(i).replace('RMB','')
def average(i):
    try:
        if(i[-1]=='-'):
            i = i[:len(i)-1]
        index = i.find('-')
        left = int(i[:index])
        right = int(i[index+1:])
        average = (left+right)/2
        return average
    except:
        return 0

In [21]:
for i in tuition_topay_unique:
    i_copy = i
    if i.find('to')!=-1:
        i = i.replace('to','-')
    if i.find(',')!=-1:
        i = remove_commas(i)
    if i.find('RMB')!=-1:
        i = remove_rmb(i)
    if i.find('-')!=-1:
        i = average(i)
    data['Tuition fees to pay'] = data['Tuition fees to pay'].replace(i_copy, int(i))

#### B - Original Tuition Fees

In [22]:
data['Original Tuition fee'] = data['Tuition Covered'] + data['Tuition fees to pay']

### PART 6 - Convert Accomodation to pay to two fields (Accomodation price, Accomodation duration)

In [23]:
len(data['Accomodation to pay'].value_counts().keys()) #62 keys 
accomodation_topay_unique = data['Accomodation to pay'].value_counts().keys()
accomodation_topay_unique

Index(['0', 'About 1200/month', 'About 1000/month', 'About 810/month',
       'About 400/month', 'About 1500/month', '0-1500/month',
       'About 400-600/month', 'About 600/month', 'About 360/month',
       'About 400-1600/month', '3,000 for one year', 'About1500/month',
       'About 450-1200/month', 'About 900/month', 'About 60-70/day',
       '1,000/month', 'About 1950/per month', 'About 120/month',
       'About 4800 /Term', '0-1200/month', '0-4,000', '2500/year', '60/day',
       '1500/month', 'About 4,000/year 0', '0- 1500/month',
       '12,000 - 36,000/year', 'About 5000RMB/per year',
       '12,000 - 36,000 /year', '700/month', '1200RMB/month', '4,000/year 0',
       'about RMB 6200/semester', 'About 1100-2000/month', '6600RMB/semester',
       '1800/year', '1200 / month', 'about RMB 450-1800', 'About 4000/year 0',
       '4000/year', '1950 0-1950RMB', 'about RMB 450-1800/month',
       'About 5000/year', 'RMB 4,000 0', 'About 1700/month', 'About 50/day',
       'Around2400/m

In [24]:
def get_value_duration(i):
    index = i.find('/')
    value = i[:index]
    duration = i[index+1:]
    return value,duration

In [25]:
temp = data['Accomodation to pay']
data['Accomodation_To_Pay'] = -1
data['Accomodation_duration'] = -1
for i in accomodation_topay_unique:
    i_copy = i
    i = str(i).upper().replace('ABOUT','')
    i = str(i).upper().replace('AROUND','')
    i = str(i).upper().replace('RMB','')
    if i.find('/')!=-1:
        value,duration = get_value_duration(i)
        data.loc[temp==i_copy,'Accomodation_To_Pay'] = value
        data.loc[temp==i_copy,'Accomodation_duration'] = duration
    elif i=='0':
        data.loc[temp==i_copy,'Accomodation_To_Pay'] = 0
        data.loc[temp==i_copy,'Accomodation_duration'] = 0
    elif i=='3,000 FOR ONE YEAR':
        data.loc[temp==i_copy,'Accomodation_To_Pay'] = 3000
        data.loc[temp==i_copy,'Accomodation_duration'] = 'YEAR'
    elif i=='0-4,000':
        data.loc[temp==i_copy,'Accomodation_To_Pay'] = 2000
        data.loc[temp==i_copy,'Accomodation_duration'] = 'YEAR'   
    elif i=='  450-1800':
        data.loc[temp==i_copy,'Accomodation_To_Pay'] = (450+1800)/2
        data.loc[temp==i_copy,'Accomodation_duration'] = 'YEAR'  
    elif i=='1950 0-1950':
        data.loc[temp==i_copy,'Accomodation_To_Pay'] = 1950
        data.loc[temp==i_copy,'Accomodation_duration'] = 'YEAR'   
    elif i==' 4,000 0':
        data.loc[temp==i_copy,'Accomodation_To_Pay'] = 4000
        data.loc[temp==i_copy,'Accomodation_duration'] = 'YEAR' 
    else:
        print(i)

In [26]:
data = data.drop('Accomodation to pay',axis=1)

#### A - Accomodation to pay

In [27]:
accomodation_topay_unique = data['Accomodation_To_Pay'].value_counts().index

In [28]:
def average(i):
    try:
        if(i[-1]=='-'):
            i = i[:len(i)-1]
        index = i.find('-')
        left = int(i[:index])
        right = int(i[index+1:])
        average = (left+right)/2
        return average
    except:
        return 0
def remove_commas(i):
    return str(i).replace(',','')

In [29]:
for i in accomodation_topay_unique:
    i_copy = i
    i = str(i).replace(str(i),str(i).strip())
    if str(i).find(',')!=-1:
        i = remove_commas(i)
    if str(i).find('-')!=-1:
        value = float(average(i))
        data['Accomodation_To_Pay'] = data['Accomodation_To_Pay'].replace(i_copy, value)
    else:
        data['Accomodation_To_Pay'] = data['Accomodation_To_Pay'].replace(i_copy, float(i))

In [30]:
data['Accomodation_To_Pay'].value_counts()

1000.0     518
0.0        437
1200.0     381
1500.0     311
810.0      240
600.0      190
400.0      189
750.0      161
500.0      134
360.0      107
3000.0     106
825.0       86
900.0       81
1950.0      72
65.0        72
4000.0      65
120.0       63
4800.0      59
2000.0      49
24000.0     46
2500.0      46
60.0        42
5000.0      26
700.0       21
6200.0      14
1550.0      14
6600.0      12
1125.0      12
1800.0      11
1700.0       2
50.0         2
2400.0       1
9750.0       1
390.0        1
958.0        1
3300.0       1
1050.0       1
6000.0       1
Name: Accomodation_To_Pay, dtype: int64

In [31]:
data[data['Accomodation_To_Pay']==-1]=0

#### B - Accomodation duration

In [32]:
accomodation_duration_unique = data['Accomodation_duration'].value_counts().index

In [33]:
data[data['Accomodation_duration']==-1]=0

In [34]:
for i in accomodation_duration_unique:
    if str(i).find('YEAR')!=-1:
        data['Accomodation_duration'] = data['Accomodation_duration'].replace(i,'YEAR')
    if str(i)=='0':
        data['Accomodation_duration'] = data['Accomodation_duration'].replace(i,'YEAR')
    if str(i).find('MONTH')!=-1:
        data['Accomodation_duration'] = data['Accomodation_duration'].replace(i,'MONTH')   
    if str(i).find('FOUR')!=-1:
        data['Accomodation_duration'] = data['Accomodation_duration'].replace(i,'MONTH') 

In [35]:
data['Accomodation_duration'].value_counts()

MONTH       2575
YEAR         799
DAY          116
TERM          59
SEMESTER      27
Name: Accomodation_duration, dtype: int64

### PART 7 - Convert Living expense to pay to two fields (living expense , living duration)

In [36]:
data['Living expense to pay'] = data['Living expense to pay'].fillna(0)

In [37]:
len(data['Living expense to pay'].value_counts().keys()) #40eys 
living_topay_unique = data['Living expense to pay'].value_counts().keys()
living_topay_unique

Index([           'About 1000/month',            'About 1500/month',
                                 '0',             'About 1500/mont',
                                   0,                   '1500/mont',
                 '1500 0- 1500/mont',                '1,600 - 2,05',
                      '0-1500/month',             'About 1000/mont',
                  '0 -1000RMB/month',            'About 1,500/mont',
              'About 1500-2000/mont',                  '1500/month',
                '2,400 - 3,000/mont',         'About 1000RMB/month',
               'About 2400/per mont',                  '1000/month',
                      '0-1000/month',        'About 2450/per month',
                       '900RMB/mont',               '0- 1000/month',
       'About 800-1000RMB/per month',                 '0-1000/mont',
                'about RMB1000/mont',        'about RMB 1000/month',
                      '1500 / month',                '900RMB /mont',
                    '2400 0-2400RM

In [38]:
def get_value_duration(i):
    index = i.find('/')
    value = i[:index]
    duration = i[index+1:]
    return value,duration

In [39]:
temp = data['Living expense to pay']
data['Expense_To_Pay'] = -1
data['Expense_duration'] = -1
for i in living_topay_unique:
    i_copy = i
    i = str(i).upper().replace('ABOUT','')
    i = str(i).upper().replace('AROUND','')
    i = str(i).upper().replace('RMB','')
    if i.find('/')!=-1:
        value,duration = get_value_duration(i)
        data.loc[temp==i_copy,'Expense_To_Pay'] = value
        data.loc[temp==i_copy,'Expense_duration'] = duration
    elif i==0 or i=='0':
        data.loc[temp==i_copy,'Expense_To_Pay'] = 0
        data.loc[temp==i_copy,'Expense_duration'] = 'MONTH'  

    
    if i=='1,600 - 2,05':
        data.loc[temp==i_copy,'Expense_To_Pay'] = 2000
        data.loc[temp==i_copy,'Expense_duration'] = 'MONTH'
    
    if i=='2400 0-2400':
        data.loc[temp==i_copy,'Expense_To_Pay'] = 1200
        data.loc[temp==i_copy,'Expense_duration'] = 'MONTH' 

#### A - Expense to pay

In [40]:
def remove_commas(i):
    return str(i).replace(',','')
def remove_rmb(i):
    return str(i).replace('RMB','')
def average(i):
    try:
        if(i[-1]=='-'):
            i = i[:len(i)-1]
        index = i.find('-')
        left = int(i[:index])
        right = int(i[index+1:])
        average = (left+right)/2
        return average
    except:
        return 0

In [41]:
expense_topay_unique = data['Expense_To_Pay'].value_counts().keys()
for i in expense_topay_unique:
    i_copy = i
    i = str(i).strip()
    if i.find(',')!=-1:
        i = remove_commas(i)
    if i.find('-')!=-1:
        i = average(i)
    else:
        i = int(i)
        
    data['Expense_To_Pay'] = data['Expense_To_Pay'].replace(i_copy,i)
#Strip the values to remove white spaces
# Remove commasr
#Find '-' and get the average

#### B - Expense_duration

In [42]:
data['Expense_duration'].value_counts()

MONTH        2713
MONT          763
PER MONTH      49
PER MONT       41
 MONTH          9
4 WEEKS         1
Name: Expense_duration, dtype: int64

In [43]:
data['Expense_duration'] = 'MONTH'

In [44]:
data = data.drop('Living expense to pay',axis=1)

# THE END

In [45]:
data.to_csv('cleaned.csv', encoding='utf-8', index=False)